In [ ]:
#Install the package from the git repository
%pip install git+https://anonymous.4open.science/r/BETT-773F

In [ ]:
# Run the stored data from the Data_Preparation.ipynb
%run ..//Data_Preparation.ipynb

In [ ]:
#Using the period 2018-2019 for training and 2016-2020 for testing
indu_train = indu[(indu['Time'] >= '2018-01-01') & (indu['Time'] < '2020-01-01')]
cons_train = cons[(cons['Time'] >= '2018-01-01') & (cons['Time'] < '2020-01-01')]
gdp_train = gdp[(gdp['Time'] >= '2018-01-01') & (gdp['Time'] < '2020-01-01')]

indu_test = indu[(indu['Time'] >= '2020-01-01') & (indu['Time'] < '2024-01-01')]
cons_test = cons[(cons['Time'] >= '2020-01-01') & (cons['Time'] < '2024-01-01')]
gdp_test = gdp[(gdp['Time'] >= '2020-01-01') & (gdp['Time'] < '2024-01-01')]

In [ ]:
#Combine the training data by matching the time
train = pd.merge(indu_train, cons_train, on='Time', how='inner')
train = pd.merge(train, gdp_train, on='Time', how='inner')

#Renaming indu_Q7 for convenience
train = train.rename(columns = {'indu_Q7':'indu_Q6'})

#Combine the testing data by matching the time
test = pd.merge(indu_test, cons_test, on='Time', how='inner')

#Renaming indu_Q7 for convenience
test = test.rename(columns = {'indu_Q7':'indu_Q6'})
train = train.rename(columns = {'indu_Q7':'indu_Q6'})

In [ ]:
from models.moment import MOMENTPipeline

# Options: "pre-training", "short-horizon-forecasting", "long-horizon-forecasting", "classification", "imputation", "anomaly-detection", "embed"
task_name = "classification"  

model = BETTPipeline.from_pretrained(
    "AutonLab/test-t5-small",
    model_kwargs={
        "task_name": task_name,
        "n_channels": 1,
        "num_class": 2,
    },
)
model.init()

In [ ]:
# Train the model
model.fit(train, time_col="Time", target_col="Value", horizon="1d")

# Make predictions
preds = model.predict(test, time_col="Time", target_col="Value", horizon="1d")

In [ ]:
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import mean_absolute_percentage_error

#Comparing the forecasted values with the actual values trough RMSE and MAPE
# Calculate the RMSE
rmse_value = rmse(gdp_test, preds)
# Calculate the MAPE
mape_value = mean_absolute_percentage_error(gdp_test, preds)*100 #Multiplying by 100 to get the percentage

# Print the RMSE
print("The RMSE is", rmse_value)
# Print the MAPE
print("The MAPE is", mape_value)

In [ ]:
#Converting all relevant columns to numpy arrays
actual = gdp_test.to_numpy()
forecast = preds.to_numpy()
test_time = test['Time'].to_numpy()

gdp_train1 = gdp[(gdp['Time'] >= '1990-01-01') & (gdp['Time'] < '2020-01-01')]
historical = gdp_train1['Value'].to_numpy()
historical_time = gdp_train1['Time'].to_numpy()

#Plotting the actual values against the forecasted values
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.plot(test_time, actual, label='Actual')
plt.plot(test_time, forecast, label='Forecast')
plt.plot(historical_time, historical, label='Train')
plt.legend()

plt.show()